[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Random.ipynb)

# 무작위 추천
0.5부터 5사이의 균등 난수를 예측 평갓값으로 한다

In [1]:
# Colab용 notebook입니다. 이 notebook 한 장에서 여러 데이터의 다운로드부터, 추천까지 완결하도록 되어 있습니다(예측 평가는 미포함)
# MovieLens 데이터를 아직 다운로드 하지 않았다면, 이 셀을 실행해서 다운로드합니다.
# MovieLens 데이터 분석은 data_download.ipynb를 참조합니다.

# 데이터 다운로드와 압축 풀기
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2022-12-27 05:40:34--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  43.4MB/s    in 1.4s    

2022-12-27 05:40:35 (43.4 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [2]:
# Movielens 데이터 로딩(데이터량이 많으므로, 로딩에 시간이 걸릴 수 있습니다)
import pandas as pd

# movieID와 제목만 사용
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genre를 list 형식으로 저장한다
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# 사용자가 부여한 영화의 태그 정보를 로딩한다
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tag를 소문자로 바꾼다
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tag를 영화별로 list 형식으로 저장한다
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# 태그 정보를 결합한다
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 평갓값 데이터만 로딩한다
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# 데이터량이 많으므로 사용자수를 1000으로 줄여서 시험해본다
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 영화 데이터와 평가 데이터를 결합한다
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 학습용과 데이터용으로 데이터를 나눈다
# 각 사용자의 최근 5건의 영화를 평가용으로 사용하고, 나머지는 학습용으로 사용한다
# 우선, 각 사용자가 평가한 영화의 순서를 계산한다
# 최근 부여한 영화부터 순서를 부여한다(1에서 시작)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
# 사용자 ID와 아이템 ID에 대해 0부터 시작하는 인덱스를 할당한다
unique_user_ids = sorted(movielens_train.user_id.unique())
unique_movie_ids = sorted(movielens_train.movie_id.unique())
user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
movie_id2index = dict(zip(unique_movie_ids, range(len(unique_movie_ids))))


In [4]:
import numpy as np
# 사용자 x 아이템 행결에서 각 셀의 예측 평갓값은 0.5~5.0의 균등 난수로 한다
pred_matrix = np.random.uniform(0.5, 5.0, (len(unique_user_ids), len(unique_movie_ids)))


In [5]:
# rmse 평가용으로 테스트 데이터에 나타나는 사용자와 아이템의 예측 평갓값을 저장한다.
movie_rating_predict = movielens_test.copy()
pred_results = []
for i, row in movielens_test.iterrows():
    user_id = row["user_id"]
    # 테스트 데이터의 아이템 ID가 학습용에 나타나지 않는 경우에도 난수를 저장한다
    if row["movie_id"] not in movie_id2index:
        pred_results.append(np.random.uniform(0.5, 5.0))
        continue
    # 테스트 데이터에 나타난 사용자 ID와 아이템 ID의 인덱스를 얻어, 평갓값 행렬의 값을 얻는다
    user_index = user_id2index[row["user_id"]]
    movie_index = movie_id2index[row["movie_id"]]
    pred_score = pred_matrix[user_index, movie_index]
    pred_results.append(pred_score)
movie_rating_predict["rating_pred"] = pred_results

In [6]:
from collections import defaultdict

# 순위 평가용 데이터를 작성한다
# 각 사용자에 대한 추천 영화는 해당 사용자가 아직 평가하지 않은 영화 중에서 10작품을 무작위로 선택한다
# 키는 사용자 ID, 값은 추천 아이템 ID 리스트
pred_user2items = defaultdict(list)
# 사용자가 이미 평가한 영화를 얻는다
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
for user_id in unique_user_ids:
    user_index = user_id2index[user_id]
    movie_indexes = np.argsort(-pred_matrix[user_index, :])
    for movie_index in movie_indexes:
        movie_id = unique_movie_ids[movie_index]
        if movie_id not in user_evaluated_movies[user_id]:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {1: [4191, 8738, 1405, 1344, 1320, 3669, 3450, 673, 1413, 6118],
             2: [56176, 4893, 761, 2072, 2951, 2540, 2294, 3596, 5362, 4664],
             3: [7880, 1592, 26082, 460, 7474, 2090, 3825, 5684, 2748, 1615],
             4: [5686, 663, 8645, 30820, 1078, 5528, 3180, 3824, 7177, 31193],
             5: [4518, 1179, 26270, 720, 3735, 4556, 29, 2307, 1974, 27912],
             6: [4800, 4234, 4682, 3255, 6808, 6954, 4840, 8501, 2630, 4082],
             7: [5056,
              1258,
              8870,
              6883,
              2695,
              7068,
              54997,
              40955,
              7299,
              33679],
             8: [5650, 4720, 6718, 2803, 5788, 6927, 4836, 4401, 2029, 525],
             9: [44937, 4541, 1928, 8872, 31, 59103, 3663, 3936, 4574, 4630],
             10: [7440, 99, 1215, 54612, 6268, 1027, 5735, 4408, 1615, 6269],
             11: [2639,
              2085,
              55269,
          

In [7]:
# user_id=2인 사용자가 학습 데이터에서 평가를 부여한 영화 목록
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [8]:
# user_id=2에 대한 추천(5656, 714, 4560)
movies[movies.movie_id.isin([5656, 714, 4560])]

,movie_id,title,genre,tag
702,714,Dead Man (1995),"[Drama, Mystery, Western]","[jim jarmusch, beautiful, iggy pop!!!, johnny ..."
4467,4560,Watchers (1988),"[Horror, Sci-Fi]","[based on a book, dean koontz]"
5558,5656,Festival in Cannes (2001),[Drama],NaN
